In [55]:
import pytreebank
import sys
import os

dataset = pytreebank.load_sst()

In [56]:
print(dataset['train'][0])

(3 (2 (2 The) (2 Rock) )(4 (3 (2 is) (4 (2 destined) (2 (2 (2 (2 (2 to) (2 (2 be) (2 (2 the) (2 (2 21st) (2 (2 (2 Century) (2 's) )(2 (3 new) (2 (2 ``) (2 Conan) )))))))(2 '') )(2 and) )(3 (2 that) (3 (2 he) (3 (2 's) (3 (2 going) (3 (2 to) (4 (3 (2 make) (3 (3 (2 a) (3 splash) )(2 (2 even) (3 greater) )))(2 (2 than) (2 (2 (2 (2 (1 (2 Arnold) (2 Schwarzenegger) )(2 ,) )(2 (2 Jean-Claud) (2 (2 Van) (2 Damme) )))(2 or) )(2 (2 Steven) (2 Segal) ))))))))))))(2 .) ))


In [57]:
out_path = os.path.join('sst_{}.txt')

In [58]:
for category in ['train', 'test', 'dev']:
    with open(out_path.format(category), 'w+') as outfile:
        for item in dataset[category]:
            outfile.write("__label__{}\t{}\n".format(
                item.to_labeled_lines()[0][0] + 1,
                item.to_labeled_lines()[0][1]
            ))
# Print the length of the training set
print(len(dataset['train']))

8544


In [1]:
import pandas as pd
from tqdm import tqdm
from tqdm import trange

In [2]:
df_train = pd.read_parquet("SST-5_train_august10.parquet")
df_val = pd.read_parquet("SST-5_validation_august10.parquet")
df_test = pd.read_parquet("SST-5_test_august10.parquet")
df_train

,label,text,vector
0,4,The Rock is destined to be the 21st Century 's...,"[0.025059903040528297, -0.0033508273772895336,..."
1,5,The gorgeously elaborate continuation of `` Th...,"[-0.010023828595876694, -0.022591764107346535,..."
2,4,Singer/composer Bryan Adams contributes a slew...,"[0.021090717986226082, 0.01804506592452526, -0..."
3,3,You 'd think by now America would have had eno...,"[0.03466595709323883, -0.028959855437278748, -..."
4,4,Yet the act is still charming here .,"[-0.019601160660386086, -0.04158126562833786, ..."
...,...,...,...
8539,1,A real snooze .,"[0.028580080717802048, 0.020179858431220055, 0..."
8540,2,No surprises .,"[-0.04816627874970436, 0.007212366443127394, 0..."
8541,4,We 've seen the hippie-turned-yuppie plot befo...,"[0.023906568065285683, 0.00044874087325297296,..."
8542,1,Her fans walked out muttering words like `` ho...,"[-0.011076121591031551, -0.011684155091643333,..."


In [3]:
from sentence_transformers import SentenceTransformer

C:\Users\Gil\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Loading pretrained GTE model for generating sentence embeddings
sentence_model = SentenceTransformer('fine_tuned_gte_august11')

# function that vectorizes a sentence
def get_sentence_embedding(text):
    if not text.strip(): 
    #.strip() gets rid of new lines
        print("Attempted to get embedding for empty text.")
        return []

    embedding = sentence_model.encode(text)

    return embedding.tolist()

In [7]:
tqdm.pandas()

In [8]:
df_train['vector'] = df_train['text'].progress_apply(get_sentence_embedding)

# replacing the old vectors with vectors that uses new fine tuned transformer model

100%|██████████| 8544/8544 [03:09<00:00, 45.09it/s]


In [9]:
df_train

,label,text,vector
0,4,The Rock is destined to be the 21st Century 's...,"[-0.0078018708154559135, -0.003602320328354835..."
1,5,The gorgeously elaborate continuation of `` Th...,"[-0.03825657069683075, -0.031692925840616226, ..."
2,4,Singer/composer Bryan Adams contributes a slew...,"[-0.011609816923737526, -0.006086165085434914,..."
3,3,You 'd think by now America would have had eno...,"[-0.020963657647371292, -0.028181064873933792,..."
4,4,Yet the act is still charming here .,"[-0.045816730707883835, -0.03720678389072418, ..."
...,...,...,...
8539,1,A real snooze .,"[0.024768028408288956, 0.01059210579842329, -0..."
8540,2,No surprises .,"[-0.021302705630660057, -0.0071574123576283455..."
8541,4,We 've seen the hippie-turned-yuppie plot befo...,"[-0.018229257315397263, -0.009151657111942768,..."
8542,1,Her fans walked out muttering words like `` ho...,"[-0.0436810702085495, -0.007883345708251, -0.0..."


In [10]:
df_val['vector'] = df_val['text'].progress_apply(get_sentence_embedding)

100%|██████████| 1101/1101 [00:26<00:00, 41.79it/s]


In [11]:
df_test['vector'] = df_test['text'].progress_apply(get_sentence_embedding)

100%|██████████| 2210/2210 [00:47<00:00, 46.09it/s]


In [12]:
df_train.to_parquet("SST-5_train_august11.parquet")
df_val.to_parquet("SST-5_validation_august11.parquet")
df_test.to_parquet("SST-5_test_august11.parquet")